In [ ]:
import numpy as np                                          
from mindquantum.core.circuit import Circuit                
from mindquantum.core.gates import H, RX, RY, RZ,X    
from mindquantum.core.parameterresolver import PRGenerator  
from mindquantum.simulator import Simulator
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split   
from mindquantum.algorithm.library import amplitude_encoder
from mindquantum.algorithm.nisq import HardwareEfficientAnsatz     
from mindquantum.core.operators import QubitOperator           # 导入QubitOperator模块，用于构造泡利算符
from mindquantum.core.operators import Hamiltonian             # 导入Hamiltonian模块，用于构建哈密顿量
import mindspore as ms                                                                         # 导入mindspore库并简写为ms
from mindquantum.framework import MQLayer                                                      # 导入MQLayer
# 导入HardwareEfficientAnsatz
from mindquantum.core.gates import RY           
import torch
from torchvision import datasets, transforms# 导入量子门RY
from scipy.ndimage import zoom

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

def filter_3_and_6(data):
    images, labels = data
    mask = (labels == 3) | (labels == 6)
    return images[mask], labels[mask]

mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
filtered_data = filter_3_and_6((mnist_dataset.data, mnist_dataset.targets))
X_data,y = filtered_data #X 图像数据 y 标签
# Compressed_X = np.array([zoom(img,0.4) for img in X])
# Compressed_X = torch.tensor(Compressed_X, dtype=torch.float32)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size=0.2, random_state=0, shuffle=True) # 将数据集划分为训练集和测试集
X_train = X_train/255
X_test = X_test/255

print(X_train.shape)                                                                                   # 打印训练集中样本的数据类型
print(X_test.shape)                                                                                    # 打印测试集中样本的数据类型

In [ ]:
X_train[0]

In [4]:
encoder, parameterResolver  = amplitude_encoder(X_train[0].numpy().flatten(),10) 

In [ ]:
encoder.summary()

In [6]:
prg = PRGenerator('alpha')
# encoder = Circuit()
sim = Simulator('mqvector', 10)
encoder, parameterResolver  = amplitude_encoder(X_train[0].numpy().flatten(),9)    
encoder = encoder.no_grad() 
# sim.apply_circuit(circuit=encoder,pr=parameterResolver)
# result = sim.get_qs()

In [ ]:
encoder.summary()

In [ ]:
ansatz = HardwareEfficientAnsatz(10, single_rot_gate_seq=[RY], entangle_gate=X, depth=1).circuit     # 通过
hams = [Hamiltonian(QubitOperator(f'Z{i}')) for i in [2, 3]]
ansatz.summary()     

In [ ]:
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")
ms.set_seed(1)                                                     # 设置生成随机数的种子
encoder = encoder.as_encoder()
ansatz  = ansatz.as_ansatz()
circuit = encoder.as_encoder() + ansatz.as_ansatz()         
sim = Simulator('mqvector', 10)
grad_ops = sim.get_expectation_with_grad(hams,
                                         circuit,
                                         parallel_worker=5)
QuantumNet = MQLayer(grad_ops)          # 搭建量子神经网络
QuantumNet


In [ ]:
from mindspore.nn import SoftmaxCrossEntropyWithLogits                         # 导入SoftmaxCrossEntropyWithLogits模块，用于定义损失函数
from mindspore.nn import Adam                                                  # 导入Adam模块用于定义优化参数
from mindspore.train import Accuracy, Model, LossMonitor                       # 导入Accuracy模块，用于评估预测准确率
import mindspore as ms
from mindspore.dataset import NumpySlicesDataset
from torch.utils.data import DataLoader# 导入NumpySlicesDataset模块，用于创建模型可以识别的数据集

loss = SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')            # 通过SoftmaxCrossEntropyWithLogits定义损失函数，sparse=True表示指定标签使用稀疏格式，reduction='mean'表示损失函数的降维方法为求平均值
opti = Adam(QuantumNet.trainable_params(), learning_rate=0.1)                  # 通过Adam优化器优化Ansatz中的参数，需要优化的是Quantumnet中可训练的参数，学习率设为0.1

model = Model(QuantumNet, loss, opti, metrics={'Acc': Accuracy()})             # 建立模型：将MindSpore Quantum构建的量子机器学习层和MindSpore的算子组合，构成一张更大的机器学习网络

X_train_for_loader = X_train.numpy().reshape(-1,28*28)
# 计算需要填充的列数
padding_size = 1022 - X_train_for_loader.shape[1]
X_train_for_loader = np.pad(X_train_for_loader, ((0, 0), (0, padding_size)), mode='constant')


# 将值 3 替换为 1
y_train[y_train == 3] = 1
# 将值 6 替换为 0
y_train[y_train == 6] = 1
y_train_for_loader = y_train.numpy()

X_test_for_loader = X_test.numpy().reshape(-1,28*28)
padding_size = 1022 - X_test_for_loader.shape[1]
X_test_for_loader = np.pad(X_test_for_loader, ((0, 0), (0, padding_size)), mode='constant')


y_test[y_test == 3] = 1
# 将值 6 替换为 0
y_test[y_test == 6] = 1
y_test_for_loader = y_test.numpy()

train_loader = NumpySlicesDataset({'features': X_train_for_loader, 'labels': y_train_for_loader}, shuffle=False).batch(5) # 通过NumpySlicesDataset创建训练样本的数据集，shuffle=False表示不打乱数据，batch(5)表示训练集每批次样本点有5个
test_loader = NumpySlicesDataset({'features': X_test_for_loader, 'labels': y_test_for_loader}).batch(5)                   # 通过NumpySlicesDataset创建测试样本的数据集，batch(5)表示测试集每批次样本点有5个


class StepAcc(ms.Callback):                                                      # 定义一个关于每一步准确率的回调函数
    def __init__(self, model, test_loader):
        self.model = model
        self.test_loader = test_loader
        self.acc = []

    def on_train_step_end(self, run_context):
        self.acc.append(self.model.eval(self.test_loader, dataset_sink_mode=False)['Acc'])


monitor = LossMonitor(20)                                                       # 监控训练中的损失，每16步打印一次损失值

acc = StepAcc(model, test_loader)                                               # 使用建立的模型和测试样本计算预测的准确率

model.train(20, train_loader, callbacks=[monitor, acc], dataset_sink_mode=False)# 将上述建立好的模型训练20次


In [14]:
y_test[y_test == 3] = 1
y_test[y_test == 6] = 0

In [ ]:
y_test[y_test == 3] = 1
y_test[y_test == 6] = 0

In [ ]:
y_test

In [ ]:
y_train[0]

In [12]:
X_train_for_loder = X_train.numpy().reshape(-1,28*28)

In [ ]:
X_train_for_loder.shape

In [ ]:
X_train_for_loader = X_train.numpy().reshape(-1, 28*28)

# 计算需要填充的列数
padding_size = 1022 - X_train_for_loader.shape[1]

# 使用 numpy.pad 进行填充
X_train_padded = np.pad(X_train_for_loader, ((0, 0), (0, padding_size)), mode='constant')
X_train_padded.shape